# Animate PyMOL session files With Jmol

Click on and drag a PyMOL session file (`.pse` file) from your computer into the file browser window to the left of this text.  
When the file is correctly dragged into the pane, a dashed,gray outline will appear and you can release your mouse button.

**TO TEST: DON'T DRAG ANYTHING IN AND JUST RUN THE NEXT CELL. IT WILL USE THE SESSION FILE ALREADY PRESENT IF YOU DON'T UPLOAD ONE. If you are trying to trouble-shoot and want the demo file included, change its name by right-clicking on the `1d66_demo.pse` demo file in the file browser pane to the left and select `Rename`.**

Run the following code cell to process the session file to make the animation.  
There are three ways to run a cell if you are not familiar with these notbeooks.

- You can run the cell by clicking on it and pressing the `run` button, shaped like a triangle heading towards the right, that is on the utility bar above this notebook.

- Click on the cell to run to select it, and then under `Run` menu above, choose `Run Selected Cells`

- Click on the cell to run to select it, and type `Shift-Enter`. Which is holding down the shift key wille pressing the enter key.


(CURRENTLY THIS IS ONLY GOING TO THE EQUIVALENT OF `rock-y-60deg.spt`.)

In [14]:
import os
import sys
import fnmatch
session_file_extension = ".pse"
session_files= []
for file in os.listdir('.'):
    if fnmatch.fnmatch(file, '*'+session_file_extension):
        session_files.append(file)
# Disregard the demo file if any other session files added
demo_file_name = "1d66_demo.pse"
if len(session_files) > 1 and demo_file_name in session_files:
    session_files.remove(demo_file_name)
#Report if demo is being used or how many session files being processed
if len(session_files) == 1 and session_files[0] == demo_file_name:
    sys.stderr.write("\nProcessing demo file '{}'"
    ".".format(demo_file_name))
else:
    sys.stderr.write("\nProcessing {} files: {}"
    ".".format(len(session_files),repr(session_files)))
for x in session_files:
    # make script to run
    script = "process.script"
    s = f'load "{x}";capture "rocking-y-60.gif" rock y 30'
    %store s > script
    !java -ea -Djava.awt.headless=true -jar /srv/conda/envs/notebook/share/jmol/JmolData.jar -g600x500 -s "{script}"


Processing demo file '1d66_demo.pse'.

Writing 's' (str) to file 'script'.
JmolData using command options ["-g600x500", "-s", "process.script"]
{headlistMaxTimeMs=60000, noConsole=true, exit=true, JmolData=true, isDataOnly=true}
Operating headless display=null nographicsallowed=false
(C) 2015 Jmol Development
Jmol Version: 14.29.15  2018-05-23 22:29
java.vendor: Java: Oracle Corporation
java.version: Java 11.0.1-internal
os.name: Linux
Access: ALL
memory: 4.2/32.4
processors available: 1
useCommandThread: false
Executing script from file: process.script
FileManager opening file /home/jovyan/process.script

FileManager.getAtomSetCollectionFromFile(1d66_demo.pse)
FileManager opening file /home/jovyan/1d66_demo.pse
The Resolver thinks PyMOL
To view raw PyMOL file data, use 'set logFile "some_filename" 
PyMOL Pickle reader cached 43 tokens; retrieved 0
PyMOL version: 130
PyMOL session file: [B@5d4bbd91
To view raw PyMOL file data, use 'set logFile "some_filename" 
cache
colors
settings
movie
selector_secrets
scene_dict
view_dic

In [8]:
import time

def executeSomething():
    #code here
    print ('.')
    time.sleep(480) #60 seconds times 8 minutes

while True:
    executeSomething()

.


KeyboardInterrupt: 